In [95]:
import dlib
import matplotlib.pyplot as pt

paths = [
    "../test-dir/2.jpeg",
    "../test-dir/a/1.jpeg",
    "../test-dir/b/c/3.jpeg"
]

In [96]:
def file_to_vectors(f):
    m1 = dlib.load_rgb_image(f)

    d1 = dlib.get_frontal_face_detector()
    d2 = dlib.shape_predictor("../models/shape_predictor_5_face_landmarks.dat")
    d3 = dlib.face_recognition_model_v1("../models/dlib_face_recognition_resnet_model_v1.dat")

    rects = d1(m1, 1)
    rs = []

    for r in rects:
        rs.append((r, d3.compute_face_descriptor(m1, d2(m1, r))))
    return rs

In [97]:
ys = []

for p in paths:
    ys.append((p, file_to_vectors(p)))

for p, y in ys:
    print(p)
    print(f"{len(y)} faces\n")
    # print(*[repr(_) for r, _ in y], '', sep='\n')

../test-dir/2.jpeg
4 faces

../test-dir/a/1.jpeg
7 faces

../test-dir/b/c/3.jpeg
7 faces



In [98]:
X = []
R = []
P = []

for p, y in ys:
    for r, x in y:
        X.append(x)
        R.append(r)
        P.append(p)


In [99]:
labels = dlib.chinese_whispers_clustering(X, 0.5)
labels

[0, 1, 2, 3, 0, 4, 3, 1, 2, 1, 5, 6, 7, 1, 8, 9, 0, 2]

In [100]:
dt = {}

for i, (x, r, p) in enumerate(zip(labels, R, P)):
    if x not in dt:
        dt[x] = []

    m = dlib.load_rgb_image(p)

    p1, p2 = r.tl_corner(), r.br_corner()
    dt[x].append(m[p1.y:p2.y, p1.x:p2.x, :])


In [101]:
import os
import cv2

for k in dt:
    dpath = f"face-outputs/{k}"
    os.makedirs(dpath, exist_ok=True)

    for j, m in enumerate(dt[k]):
        cv2.imwrite(os.path.join(dpath, f"{j}.png"), m[:,:,[2,1,0]])
